In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astropy.table import Table, hstack, vstack
from sklearn.neighbors import KernelDensity
from tqdm import tqdm
import astropy.units as u
from astropy.coordinates import SkyCoord
import mpl_scatter_density

import numpy as np
from matplotlib import cm
from matplotlib.patches import Rectangle
from matplotlib.colors import ListedColormap


In [ ]:
params = {
    "legend.fontsize": "x-large",
    "axes.labelsize": "x-large",
    "axes.titlesize": "x-large",
    "xtick.labelsize": "x-large",
    "ytick.labelsize": "x-large",
    "figure.facecolor": "w",
    "xtick.top": True,
    "ytick.right": True,
    "xtick.direction": "in",
    "ytick.direction": "in",
    "font.family": "serif",
    "mathtext.fontset": "dejavuserif"
}
plt.rcParams.update(params)

def cmap_white(cmap_name):
    """Returns a colormap with white as the lowest value color."""
    import numpy as np
    try:
        from matplotlib import cm
        from matplotlib.colors import ListedColormap
        cmap = cm.get_cmap(cmap_name, 256)
    except ValueError:
        import seaborn as sns
        cmap = sns.color_palette("flare", as_cmap=True)
    newcolors = cmap(np.linspace(0, 1, 256))
    white = np.array([1, 1, 1, 0])
    newcolors[:1, :] = white
    cmap_white = ListedColormap(newcolors)
    return cmap_white

In [ ]:
base_path = Path("/global/cfs/cdirs/desi/users/bid13/DESI_II/")
patch = "COSMOS"
hsc_path = base_path / "target_data"/ f"HSC_{patch}_I_mag_lim_24.8.fits"

In [ ]:
def flux_to_mag(flux):
    return -2.5*np.log10(flux*1e-9) + 8.90

In [ ]:
hsc_cat = Table.read(hsc_path).to_pandas()
hsc_cat["i_mag"] = flux_to_mag(hsc_cat["i_cmodel_flux"])-hsc_cat["a_i"]
hsc_cat["r_mag"] = flux_to_mag(hsc_cat["r_cmodel_flux"])-hsc_cat["a_r"]
hsc_cat["z_mag"] = flux_to_mag(hsc_cat["z_cmodel_flux"])-hsc_cat["a_z"]
hsc_cat["g_mag"] = flux_to_mag(hsc_cat["g_cmodel_flux"])-hsc_cat["a_g"]

hsc_cat["g_fiber_mag"] = flux_to_mag(hsc_cat["g_fiber_flux"])-hsc_cat["a_g"]
hsc_cat["i_fiber_mag"] = flux_to_mag(hsc_cat["i_fiber_flux"])-hsc_cat["a_i"]
hsc_cat["r_fiber_mag"] = flux_to_mag(hsc_cat["r_fiber_flux"])-hsc_cat["a_r"]
hsc_cat["z_fiber_mag"] = flux_to_mag(hsc_cat["z_fiber_flux"])-hsc_cat["a_z"]

hsc_cat["i_mag_psf"] = flux_to_mag(hsc_cat["i_psfflux_flux"])-hsc_cat["a_i"]
# hsc_cat["i_fiber_tot_mag"] = flux_to_mag(hsc_cat["i_fiber_tot_flux"])-hsc_cat["a_i"]

In [ ]:
## Quality cuts
# valid I-band flux
qmask = np.isfinite(hsc_cat["i_cmodel_flux"]) & (hsc_cat["i_cmodel_flux"]>0)
#cmodel fit not failed
qmask &= (~hsc_cat["i_cmodel_flag"].values)
#General Failure Flag
qmask &= (~hsc_cat["i_sdsscentroid_flag"].values)


# Possible cuts: Bright objects nearby, bad pixels

#star-galaxy separation (is point source in I band)
extendedness = hsc_cat["i_mag_psf"]-hsc_cat["i_mag"]
mask_ext = (hsc_cat["i_mag"]>23) | (extendedness>0.02)


i_min = 18 #potentially 20
i_max = 24.5
i_mask = (hsc_cat["i_mag"] <i_max) & (hsc_cat["i_mag"] >i_min)

#We have decided to not have any color cut or fiber magnitude cut
# i_fiber_min = 18
# i_fiber_max = 25 
# mask &= (hsc_cat["i_fiber_mag"] <i_fiber_max) & (hsc_cat["i_fiber_mag"] >i_fiber_min)

plot i-mage vs i-fiber-mag distribution for the parent sample

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5,5),subplot_kw={"projection":"scatter_density"})
ax.scatter_density(hsc_cat[qmask]["i_mag"], hsc_cat[qmask]["i_fiber_mag"],cmap=cmap_white("viridis"),dpi=1000)
ax.set_aspect('equal')
x = np.linspace(15,25,100)
ax.plot(x,x, ls="--", c="k", alpha=0.5)
ax.set_xlim(16,26)
ax.set_ylim((16,26))
ax.set_xlabel("$i$ mag")
ax.set_ylabel("$i$ fiber mag")

Plot i-mag distribution of various other DESI samples

In [ ]:
plt.hist(hsc_cat["i_mag"][qmask],bins=50, histtype="step", label="LSST Y1")
plt.hist(hsc_cat["i_mag"][qmask & (hsc_cat["z_mag"]<20.8)],bins=50, histtype="step", label = "~DC3R2")
plt.hist(hsc_cat["i_mag"][qmask & (hsc_cat["r_mag"]<19.5)],bins=50, histtype="step", label = "~BGS")
plt.hist(hsc_cat["i_mag"][qmask & (hsc_cat["r_mag"]<20.1)],bins=50, histtype="step", label = "~BGS Faint")
plt.hist(hsc_cat["i_mag"][qmask & (hsc_cat["g_mag"]<23.4)],bins=50, histtype="step", label = "~ELG")
plt.hist(hsc_cat["i_mag"][qmask & (hsc_cat["z_mag"]<21)],bins=50, histtype="step", label = "~LRG")
for i in np.linspace(18,24.5,14):
    plt.axvline(i,c="k",ls="--",alpha=0.2)
plt.xlabel("$i$-mag", fontsize=20)
plt.yscale("log")
plt.legend(loc="upper left", frameon=False)

Star-Galaxy separation

In [ ]:
extendedness = hsc_cat["i_mag_psf"]-hsc_cat["i_mag"]
mask_ext = (hsc_cat["i_mag"]>23) | (extendedness>0.02)



qmask_gr = np.isfinite(hsc_cat["g_cmodel_flux"]) & (hsc_cat["g_cmodel_flux"]>0)
qmask_gr &= np.isfinite(hsc_cat["r_cmodel_flux"]) & (hsc_cat["r_cmodel_flux"]>0)
#cmodel fit not failed
qmask_gr &= (~hsc_cat["g_cmodel_flag"].values)
qmask_gr &= (~hsc_cat["r_cmodel_flag"].values)
#General Failure Flag
qmask_gr &= (~hsc_cat["g_sdsscentroid_flag"].values)
qmask_gr &= (~hsc_cat["r_sdsscentroid_flag"].values)


sels_cat = hsc_cat[qmask & i_mask & qmask_gr & mask_ext]
sels_cat = sels_cat.reset_index()

gr = sels_cat["g_mag"] - sels_cat["r_mag"]
ri = sels_cat["r_mag"] - sels_cat["i_mag"]

value_mask = (gr>-1) & (gr<3) & (ri>-0.5) & (ri<2.5)

fig, ax = plt.subplots(1,1,subplot_kw={"projection":"scatter_density"})
ax.scatter_density(gr[value_mask], ri[value_mask],cmap=cmap_white("viridis"),dpi=100)
ax.set_xlabel("$g-r$")
ax.set_ylabel("$r-i$")

Select the final targets and resample to a uniform distribution in i-mag

In [ ]:
hist, bin_edges = np.histogram(sels_cat["i_mag"], bins=10000,density=True)
bin_edges[-1] = i_max
bin_edges[0] = i_min

In [ ]:
bin_membership = np.digitize(sels_cat["i_mag"], bin_edges)

In [ ]:
prob = hist[bin_membership-1]

weights = 1/prob
weights = weights/weights.sum()

In [ ]:
rng = np.random.default_rng(seed=42)
resample_idx = rng.choice(np.arange(len(sels_cat["i_mag"])), size=25000, replace=False, p=weights)
final_sel = sels_cat.iloc[resample_idx]

# sample_weights = weights/weights.max()
# mymask = rng.uniform(size=len(sample_weights)) < sample_weights
# final_sel = sels_cat[mymask]

In [ ]:
plt.figure(figsize=(7,5))
plt.hist(sels_cat["i_mag"],bins=100, histtype="step", label="Magnitude Limited Sample",lw=2)
plt.hist(final_sel["i_mag"],bins=100, histtype="step",  label="Uniform Magnitude Sample",lw=2)

# plt.hist(sels_cat["i_mag"][resample_idx][np.isfinite(sels_cat["specz_redshift"][resample_idx])],bins=50, histtype="step", density=True, label="Known Redshifts")
plt.axhline(np.mean(np.histogram(sels_cat["i_mag"][resample_idx],bins=100, density=True)[0]), c="k", ls="--")
plt.xlabel("$i$-band Magnitude", fontsize=20)
plt.ylabel("Normalized Frequency", fontsize=20)
plt.yscale("log")
plt.legend(loc="upper left",frameon=False)
# plt.savefig("resampled_distribution.pdf", bbox_inches="tight")

In [ ]:
# small_cat = hsc_cat.sample(n=10000)
fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.scatter(sels_cat.loc[resample_idx, "ra"],sels_cat.loc[resample_idx, "dec"],marker=".",s=1, alpha=1)
# plt.scatter(small_cat["ra"],small_cat["dec"],marker=".",s=1)
# plt.scatter(redm["RA"],redm["DEC"],marker=".",s=5, c="r")
ra_min = 148
ra_max = 152
dec_min = 0
dec_max = 4

ax.set_xlabel("RA $\degree$", fontsize=20)
ax.set_ylabel("DEC $\degree$", fontsize=20)
ax.set_xticks(np.arange(ra_min, ra_max+1, 1))
ax.set_xticks(np.arange(ra_min, ra_max+1, 0.5), minor=True)
ax.set_yticks(np.arange(dec_min, dec_max+1, 1))
ax.set_yticks(np.arange(dec_min, dec_max+1, 0.5), minor=True)
ax.set_xlim(ra_max+0.2, ra_min-0.2)
ax.set_ylim(dec_min-0.2, dec_max+0.2)
# And a corresponding grid
ax.grid(which='both')

# Or if you want different settings for the grids:
ax.grid(which='minor', alpha=0.5)
ax.grid(which='major', alpha=0.8)
ax.set_title(f"Target Density: {len(final_sel)/16} per sq deg.")

In [ ]:
TEST_NAME = "TEST1"
blank = np.zeros(len(final_sel))
Table({"RA":final_sel["ra"], "DEC":final_sel["dec"], "PMRA":blank, "PMDEC":blank,
       'REF_EPOCH':blank+2000, "OVERRIDE":(blank+1).astype(bool),"I_MAG": final_sel["i_mag"]}, ).write(base_path / f"{TEST_NAME}_{patch}_LSSTY1_target_list.fits", overwrite=True)

### Explore the fiberassign files

In [ ]:
# fa_path = Path("/global/cfs/cdirs/desi/survey/fiberassign/special/tertiary/0015/tertiary-targets-0015-assign.fits")

In [ ]:
# fa = Table.read(fa_path, hdu=1)
# names = [name for name in fa.colnames if len(fa[name].shape) <= 1]
# fa = fa[names].to_pandas()

In [ ]:
# fa = fa[fa["TERTIARY_TARGET"].isin([b"LSST_WLY1_HIP", b"LSST_WLY1_LOP"])]
# fa = fa[fa["NASSIGN"]>0]

In [ ]:
# plt.hist(fa["NASSIGN"], bins=np.arange(0,26), histtype="step")
# plt.ylabel("Count")
# plt.xlabel("Number of exposures (1000s)")
# # plt.yscale("log")

In [ ]:
Table.from_pandas(fa).write("LSST_WLY1_XMM_LSS_fiber_assigned.fits")